<a href="https://colab.research.google.com/github/salvapineda/notebooks/blob/main/NetworkEconomicDispatch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Network Economic Dispatch 

This Jupyter Notebook solves an instance of the Network Constrained Economic Dispatch Problem described in XXX

## Requirements

In [1]:
!pip install -q pyomo
!apt-get install -y -qq glpk-utils
import pyomo.environ as pe
import pandas as pd
import numpy as np
glpk = pe.SolverFactory('glpk', executable='/usr/bin/glpsol')

## Input Data

In [34]:
gen = pd.DataFrame({
       'unit': [1,   2], 
       'bus':  [1,   2],
       'cost': [10,  20], 
       'pmin': [0,   0], 
       'pmax': [100, 100]})
lin = pd.DataFrame({
       'line': [1,   2,   3], 
       'from': [1,   1,   2],
       'to':   [2,   3,   3], 
       'sus':  [0.5, 0.5, 0.5], 
       'cap':  [25,  100, 100]})
dem = pd.DataFrame({
       'dem':  [1  ], 
       'bus':  [3  ],
       'level':[100]})
ngen = len(gen)
nbus = max(max(lin['from']),max(lin['to']))
nlin = len(lin)
ndem = len(dem)

## Determining the generator-bus and demand-bus incidence matrixes

In [35]:
gen_bus = np.zeros((ngen,nbus))
for g in range(ngen):
  gen_bus[g,gen.loc[g,'bus']-1]=1
dem_bus = np.zeros((ndem,nbus))
for g in range(ndem):
  dem_bus[g,dem.loc[g,'bus']-1]=1

## Computing Power Transfer Distribution Factors

In [36]:
lin_bus = np.zeros((nlin,nbus))
for l in range(nlin):
  lin_bus[l,lin.loc[l,'from']-1]=1
  lin_bus[l,lin.loc[l,'to']-1]=-1    
matrixX_inv = np.diag(1/lin['sus'])
matrixA = np.delete(np.array(lin_bus),0,axis=1)
matrixB = np.linalg.multi_dot([matrixA.T,matrixX_inv,matrixA]) 
ptdf = np.linalg.multi_dot([matrixX_inv,matrixA,np.linalg.inv(matrixB)])
ptdf = np.round(np.insert(ptdf,0,np.zeros((nlin)),axis=1),5)



## Solving the Network Constrained Economic Dispatch

In [37]:
# Model
m = pe.ConcreteModel()
# Sets
m.g = pe.Set(initialize=list(range(ngen)))
m.j = pe.Set(initialize=list(range(ndem)))
m.n = pe.Set(initialize=list(range(nbus)))
m.l = pe.Set(initialize=list(range(nlin)))
# Variables
m.p = pe.Var(m.g,within=pe.NonNegativeReals)
def obj_rule(m):
  return sum(gen.loc[g,'cost']*m.p[g] for g in m.g)
m.obj = pe.Objective(rule=obj_rule)
# Energy balance
def bal_rule(m):
  return sum(m.p[g] for g in m.g) == sum(dem.loc[j,'level'] for j in m.j)
m.bal = pe.Constraint(rule=bal_rule)
# Maximum generation
def max_gen_rule(m,g):
  return m.p[g] <= gen.loc[g,'pmax']
m.max_gen = pe.Constraint(m.g, rule=max_gen_rule)
# Maximum power flow
def max_flow_rule(m,l):
  return sum(ptdf[l,n]*(sum(gen_bus[g,n]*m.p[g] for g in m.g) - sum(dem_bus[j,n]*dem.loc[j,'level'] for j in m.j)) for n in m.n) >= -lin.loc[l,'cap'] 
m.max_flow = pe.Constraint(m.l, rule=max_flow_rule)
# Minimum power flow
def min_flow_rule(m,l):
  return sum(ptdf[l,n]*(sum(gen_bus[g,n]*m.p[g] for g in m.g) - sum(dem_bus[j,n]*dem.loc[j,'level'] for j in m.j)) for n in m.n) <= lin.loc[l,'cap'] 
m.min_flow = pe.Constraint(m.l, rule=min_flow_rule)
# We solve the optimization problem using GLPK
glpk.solve(m).write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 1124.99437502812
  Upper bound: 1124.99437502812
  Number of objectives: 1
  Number of constraints: 10
  Number of variables: 3
  Number of nonzeros: 11
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.013015270233154297
# ----------------------------------------------------------
#   Solution Information
# ------------

## Solution Output

In [38]:
# Print results
print('obj =',m.obj())
print('p1 =',m.p[0].value)
print('p2 =',m.p[1].value)

obj = 1124.994375028125
p1 = 87.5005624971875
p2 = 12.4994375028125
